In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import json
import pandas as pd
import traceback

sys.path.append("../../../") # add results folder as a python path 
from viz import *
from analyze import *

In [2]:
path = os.path.dirname("./data2/BOHB/")
target_dirs = os.listdir(path)

In [12]:
def getTrialResult(result):
    cum_exec_time = []
    cum_opt_time = []
    cum_first = True

    exec_times = []
    opt_times = []
    epochs = []
    model_info = []
    errors = []
    accuracies = []

    for r in result:
        dict_r = json.loads(r)
#         print(dict_r)
        exec_time = dict_r[2]['finished'] - dict_r[2]['started']
        opt_time = dict_r[2]['finished'] - dict_r[2]['submitted']

        if cum_first is True:
            cum_exec_time_last_value = 0.0
            cum_opt_time_last_value = 0.0
            cum_first = False

        if cum_exec_time_last_value <= 86400:
            exec_times.append(exec_time)
            opt_times.append(opt_time)
            cum_exec_time.append((cum_exec_time_last_value + exec_time))
            cum_opt_time.append((cum_opt_time_last_value + opt_time))
            epochs.append(dict_r[1])
            model_info.append(dict_r[0])
            if dict_r[3] is not None:
                errors.append(dict_r[3]['loss'])
                accuracies.append(dict_r[3]['info']['test accuracy'])
            else:
                errors.append(1.0)
                accuracies.append(0.0)
            
            cum_exec_time_last_value = cum_exec_time[len(cum_exec_time)-1]
            cum_opt_time_last_value = cum_opt_time[len(cum_opt_time)-1]
            
            dict_result = dict([("cum_exec_time", cum_exec_time),
                        ("exec_time", exec_times),
                        ("opt_time", opt_times),
                        ("model_info", model_info),
                        ("cum_opt_time", cum_opt_time),
                        ("error", errors),
                        ("accuracy", accuracies),
                        ("epoch", epochs)])
        else:
            continue
        
    return dict_result

In [4]:
def getTrialConfig(config):
    config_dict = dict()
    
    for c in config:
        dict_c = json.loads(c)
        target = dict_c[2]
        config_dict[str(dict_c[0])] = target
        
    return config_dict

In [5]:
def addModelInfo(trial_result, trial_config):
    model_index_list = []
    for i in range(len(trial_result['model_info'])):
        model_idx = str(trial_result['model_info'][i])
        model_info = trial_config[model_idx]
        target_index = -2
        if 'lookup_index' in model_info:
            target_index = model_info['lookup_index']
        elif trial_result['error'][i] == 1:
            target_index = 0
        model_index_list.append(target_index)
    trial_result['model_index'] = model_index_list
    del trial_result['model_info']

In [16]:
bohb_results = dict()
bohb_configs = dict()

for d in target_dirs:
#     print(d)
    if d != 'iter10_result':
        continue
    else:
        trial_num = '0'
#     trial_num = d[5:]
    
    with open(path+'/'+d+'/'+'results.json') as rf:
        result = rf.readlines()

    result = [x.strip() for x in result]
    result_dict=getTrialResult(result)
    
    with open(path+'/'+d+'/'+'configs.json') as rf:
        config = rf.readlines()
        
    config = [x.strip() for x in config]
#     bohb_configs[trial_num]=getTrialConfig(config)
    config_dict=getTrialConfig(config)
    
    addModelInfo(result_dict, config_dict)
    
    bohb_results[trial_num]=result_dict
    

In [17]:
with open('iter10_result.json', 'w') as outfile:
    json.dump(bohb_results, outfile)

In [18]:
bohb_results['0']

{'cum_exec_time': [13.027785539627075,
  21.68964457511902,
  30.39171528816223,
  30.498034238815308,
  39.10349369049072,
  47.64689326286316,
  56.30432677268982,
  64.88485145568848,
  64.98066449165344,
  88.1446487903595,
  111.31443786621094,
  134.45494866371155,
  194.04528522491455,
  194.14853930473328,
  217.4315025806427,
  240.69292855262756,
  300.35132908821106,
  359.9873192310333,
  360.0931670665741,
  360.19276547431946,
  368.9044551849365,
  377.4766356945038,
  386.04608392715454,
  394.63341641426086,
  403.21159839630127,
  403.30673360824585,
  403.4132881164551,
  403.5186610221863,
  403.6156690120697,
  426.9301691055298,
  450.1286025047302,
  473.3085832595825,
  532.9218072891235,
  556.112268447876,
  579.2882249355316,
  579.3827002048492,
  639.1015737056732,
  699.0116283893585,
  758.7362463474274,
  758.8408303260803,
  767.43541264534,
  776.0044820308685,
  784.5621151924133,
  793.1357443332672,
  801.7326016426086,
  801.8356165885925,
  810.41